In [ ]:
import pandas as pd
from caas_jupyter_tools import display_dataframe_to_user

path = "dataset\olist_closed_deals_dataset.csv"
df = pd.read_csv(path)

# Basic info
shape = df.shape
cols = list(df.columns)
head = df.head(10)

na = df.isna().sum().sort_values(ascending=False)
na_pct = (na / len(df) * 100).round(2)

summary_na = pd.DataFrame({"missing": na, "missing_pct": na_pct}).reset_index().rename(columns={"index":"column"})
display_dataframe_to_user("olist_closed_deals_dataset — sample", head)
display_dataframe_to_user("Missing values by column", summary_na)

(shape, cols, df.dtypes)



In [ ]:
import pandas as pd
import numpy as np
from caas_jupyter_tools import display_dataframe_to_user

df = pd.read_csv("/mnt/data/olist_closed_deals_dataset.csv")

# Parse won_date
df["won_date"] = pd.to_datetime(df["won_date"], errors="coerce")

# Basic time range
time_min = df["won_date"].min()
time_max = df["won_date"].max()
won_missing = df["won_date"].isna().sum()

# Uniques
uniques = {c: df[c].nunique(dropna=True) for c in ["mql_id","seller_id","sdr_id","sr_id"]}
uniques, time_min, time_max, won_missing



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from caas_jupyter_tools import display_dataframe_to_user

df = pd.read_csv("/mnt/data/olist_closed_deals_dataset.csv")
df["won_date"] = pd.to_datetime(df["won_date"], errors="coerce")
df["won_month"] = df["won_date"].dt.to_period("M").astype(str)

# Deals per month
monthly = df["won_month"].value_counts().sort_index().rename_axis("month").reset_index(name="closed_deals")
display_dataframe_to_user("Closed deals per month", monthly)

plt.figure()
plt.bar(monthly["month"], monthly["closed_deals"])
plt.title("Closed deals per month (won_date)")
plt.xlabel("Month")
plt.ylabel("Closed deals")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Top categories
def top_table(col, n=15):
    vc = df[col].value_counts(dropna=False).head(n)
    out = vc.rename_axis(col).reset_index(name="count")
    out["share_pct"] = (out["count"]/len(df)*100).round(2)
    return out

top_business_segment = top_table("business_segment", 15)
top_lead_type = top_table("lead_type", 15)
top_behaviour = top_table("lead_behaviour_profile", 15)
top_business_type = top_table("business_type", 15)

display_dataframe_to_user("Top business_segment", top_business_segment)
display_dataframe_to_user("Top lead_type", top_lead_type)
display_dataframe_to_user("Top lead_behaviour_profile", top_behaviour)
display_dataframe_to_user("Top business_type", top_business_type)



In [ ]:
import pandas as pd
import numpy as np
from caas_jupyter_tools import display_dataframe_to_user

df = pd.read_csv("/mnt/data/olist_closed_deals_dataset.csv")
df["won_date"] = pd.to_datetime(df["won_date"], errors="coerce")

# Binary fields distribution
binary_cols = ["has_company","has_gtin"]
bin_tables = {}
for c in binary_cols:
    vc = df[c].value_counts(dropna=False)
    tab = vc.rename_axis(c).reset_index(name="count")
    tab["share_pct"] = (tab["count"]/len(df)*100).round(2)
    bin_tables[c] = tab
display_dataframe_to_user("has_company distribution", bin_tables["has_company"])
display_dataframe_to_user("has_gtin distribution", bin_tables["has_gtin"])

# declared_product_catalog_size stats
num_cols = ["declared_product_catalog_size","declared_monthly_revenue"]
stats = []
for c in num_cols:
    s = df[c]
    stats.append({
        "column": c,
        "n": int(s.notna().sum()),
        "missing": int(s.isna().sum()),
        "min": float(s.min()) if s.notna().any() else None,
        "p25": float(s.quantile(0.25)) if s.notna().any() else None,
        "median": float(s.median()) if s.notna().any() else None,
        "p75": float(s.quantile(0.75)) if s.notna().any() else None,
        "max": float(s.max()) if s.notna().any() else None,
        "mean": float(s.mean()) if s.notna().any() else None,
    })
stats_df = pd.DataFrame(stats)
display_dataframe_to_user("Numeric fields summary", stats_df)



In [ ]:
import pandas as pd
import numpy as np
from caas_jupyter_tools import display_dataframe_to_user

df = pd.read_csv("/mnt/data/olist_closed_deals_dataset.csv")
df["won_date"] = pd.to_datetime(df["won_date"], errors="coerce")

# declared_monthly_revenue distribution (since many zeros)
rev = df["declared_monthly_revenue"]
rev_zero = (rev == 0).sum()
rev_pos = (rev > 0).sum()
rev_neg = (rev < 0).sum()
rev_missing = rev.isna().sum()

# quantiles for positive only
rev_pos_series = rev[rev > 0]
quantiles = rev_pos_series.quantile([0.5, 0.75, 0.9, 0.95, 0.99]).to_frame("value").reset_index().rename(columns={"index":"quantile"})

display_dataframe_to_user("Revenue positives — key quantiles", quantiles)

{"rows": len(df), "rev_zero": int(rev_zero), "rev_pos": int(rev_pos), "rev_neg": int(rev_neg), "rev_missing": int(rev_missing)}



In [ ]:
import pandas as pd
from caas_jupyter_tools import display_dataframe_to_user

df = pd.read_csv("/mnt/data/olist_closed_deals_dataset.csv")
df["won_date"] = pd.to_datetime(df["won_date"], errors="coerce")

# cross tab: business_type vs has_company/has_gtin
ct_company = pd.crosstab(df["business_type"], df["has_company"], dropna=False)
ct_gtin = pd.crosstab(df["business_type"], df["has_gtin"], dropna=False)

display_dataframe_to_user("Crosstab business_type x has_company", ct_company.reset_index())
display_dataframe_to_user("Crosstab business_type x has_gtin", ct_gtin.reset_index())



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

df = pd.read_csv("/mnt/data/olist_closed_deals_dataset.csv")
df["won_date"] = pd.to_datetime(df["won_date"], errors="coerce")
df["won_month"] = df["won_date"].dt.to_period("M").astype(str)

out_dir = Path("/mnt/data/olist_closed_deals_analysis")
out_dir.mkdir(parents=True, exist_ok=True)

# Tables
monthly = df["won_month"].value_counts().sort_index().rename_axis("month").reset_index(name="closed_deals")
monthly.to_csv(out_dir / "closed_deals_per_month.csv", index=False)

def top_table(col, n=20):
    vc = df[col].value_counts(dropna=False).head(n)
    out = vc.rename_axis(col).reset_index(name="count")
    out["share_pct"] = (out["count"]/len(df)*100).round(2)
    return out

top_business_segment = top_table("business_segment", 30)
top_lead_type = top_table("lead_type", 30)
top_behaviour = top_table("lead_behaviour_profile", 30)
top_business_type = top_table("business_type", 10)

top_business_segment.to_csv(out_dir / "top_business_segment.csv", index=False)
top_lead_type.to_csv(out_dir / "top_lead_type.csv", index=False)
top_behaviour.to_csv(out_dir / "top_lead_behaviour_profile.csv", index=False)
top_business_type.to_csv(out_dir / "top_business_type.csv", index=False)

# Binary distributions
for c in ["has_company", "has_gtin"]:
    vc = df[c].value_counts(dropna=False)
    tab = vc.rename_axis(c).reset_index(name="count")
    tab["share_pct"] = (tab["count"]/len(df)*100).round(2)
    tab.to_csv(out_dir / f"dist_{c}.csv", index=False)

# Numeric stats
num_cols = ["declared_product_catalog_size","declared_monthly_revenue"]
stats = []
for c in num_cols:
    s = df[c]
    stats.append({
        "column": c,
        "n": int(s.notna().sum()),
        "missing": int(s.isna().sum()),
        "min": float(s.min()) if s.notna().any() else None,
        "p25": float(s.quantile(0.25)) if s.notna().any() else None,
        "median": float(s.median()) if s.notna().any() else None,
        "p75": float(s.quantile(0.75)) if s.notna().any() else None,
        "max": float(s.max()) if s.notna().any() else None,
        "mean": float(s.mean()) if s.notna().any() else None,
    })
pd.DataFrame(stats).to_csv(out_dir / "numeric_summary.csv", index=False)

# Charts
plt.figure()
plt.bar(monthly["month"], monthly["closed_deals"])
plt.title("Closed deals per month (won_date)")
plt.xlabel("Month")
plt.ylabel("Closed deals")
plt.xticks(rotation=90)
plt.tight_layout()
p1 = out_dir / "closed_deals_per_month.png"
plt.savefig(p1, dpi=200)
plt.close()

# Pareto for business_segment (if many)
bs = df["business_segment"].fillna("NaN").value_counts().reset_index()
bs.columns = ["business_segment", "count"]
bs["cum_share_pct"] = (bs["count"].cumsum() / bs["count"].sum() * 100).round(2)
plt.figure()
plt.bar(bs["business_segment"].head(20), bs["count"].head(20))
plt.title("Top 20 business_segment (bars) + cumulative share (line)")
plt.xlabel("business_segment")
plt.ylabel("Closed deals")
plt.xticks(rotation=90)
ax = plt.gca()
ax2 = ax.twinx()
ax2.plot(range(min(20, len(bs))), bs["cum_share_pct"].head(20))
ax2.set_ylabel("Cumulative share (%)")
plt.tight_layout()
p2 = out_dir / "business_segment_pareto_top20.png"
plt.savefig(p2, dpi=200)
plt.close()

# Summary text
time_min = df["won_date"].min()
time_max = df["won_date"].max()
uniques = {c: int(df[c].nunique(dropna=True)) for c in ["mql_id","seller_id","sdr_id","sr_id"]}
rev = df["declared_monthly_revenue"]
summary = {
    "rows": len(df),
    **{f"unique_{k}": v for k, v in uniques.items()},
    "won_date_min": str(time_min),
    "won_date_max": str(time_max),
    "declared_monthly_revenue_zero_count": int((rev==0).sum()),
    "declared_monthly_revenue_positive_count": int((rev>0).sum()),
    "declared_product_catalog_size_nonnull": int(df["declared_product_catalog_size"].notna().sum()),
}
pd.DataFrame([summary]).to_csv(out_dir / "summary.csv", index=False)

[str(out_dir), str(p1), str(p2), str(out_dir/"summary.csv")]



In [ ]:
from pathlib import Path
import zipfile, os

folder = Path("/mnt/data/olist_closed_deals_analysis")
zip_path = Path("/mnt/data/olist_closed_deals_analysis_outputs.zip")

# Recreate zip
if zip_path.exists():
    zip_path.unlink()

with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for p in folder.rglob("*"):
        if p.is_file():
            z.write(p, arcname=p.relative_to(folder))

str(zip_path), sorted([p.name for p in folder.iterdir()])[:10], len(list(folder.rglob("*")))

